## Использование API clearspending

Подробнее тут: <https://github.com/datacoon/clearspending-examples/wiki>

Ещё презентация про API: <https://issuu.com/teplitsa_st/docs/api_pres>

## Сначала импортируем библиотеки + функция скачивания сырых данных с сайта

In [101]:
import json
import requests
import datetime
import math
from dateutil import parser
import pandas as pd

pd.set_option("max_colwidth", None)

def query_clearspending(base_url):
    print("Запрашиваем 1 страницу")
    results = []
    f = requests.get(base_url)
    contents = json.loads(f.text)
    results.extend(contents["contracts"]["data"])

    total = contents["contracts"]["total"]
    perpage = contents["contracts"]["perpage"]
    pages = math.ceil(total / perpage)
    
    print("Всего", total, "контрактов")
    if pages >= 2:
        for p in range(2, pages+1):
            print("Страница {0}/{1}".format(p, pages))
            f = requests.get(base_url + "&page=" + str(p))
            contents = json.loads(f.text)
            results.extend(contents["contracts"]["data"])
            
    return results

## Вот здесь поиск контрактов по ИНН

In [102]:
inn_to_search = [
    3808131271, # АДМИНИСТРАЦИЯ ГОРОДА ИРКУТСКА
    3808224350, # ДУМА ГОРОДА ИРКУТСКА
    3808171394, # АППАРАТ ГУБЕРНАТОРА ИРКУТСКОЙ ОБЛАСТИ И ПРАВИТЕЛЬСТВА ИРКУТСКОЙ ОБЛАСТИ
    3800000608, # ЗАКОНОДАТЕЛЬНОЕ СОБРАНИЕ ИРКУТСКОЙ ОБЛАСТИ
    3808131923, # МИНИСТЕРСТВО ПО РЕГУЛИРОВАНИЮ КОНТРАКТНОЙ СИСТЕМЫ В СФЕРЕ ЗАКУПОК ИРКУТСКОЙ ОБЛАСТИ
]

url_tpl = "http://openapi.clearspending.ru/restapi/v3/contracts/search/?customerinn={0}&daterange=01.02.2021-{1}"
now = datetime.datetime.now().strftime("%d.%m.%Y")

contracts = []
for inn in inn_to_search:
    url = url_tpl.format(inn, now)
    cs = query_clearspending(url)
    contracts.extend(cs)

Запрашиваем 1 страницу
Всего 2 контрактов
Запрашиваем 1 страницу
Всего 2 контрактов
Запрашиваем 1 страницу
Всего 1 контрактов
Запрашиваем 1 страницу
Всего 11 контрактов
Запрашиваем 1 страницу
Всего 1 контрактов


## Раскидываем нужные данные контракта в таблицу, сортируем по времени и показываем

In [98]:
ds = []
titles = ["Дата публикации", "ФЗ", "Цена, ₽", "Услуги", "Покупатель", "Подробнее"]

for contract in contracts:
    ds.append((parser.parse(contract["publishDate"]), contract["fz"], math.floor(contract["price"]),\
               ",".join((p["name"] for p in contract["products"])),\
               contract["customer"]["fullName"], contract["printFormUrl"]))

ds.sort(key = lambda row: row[0], reverse=True)
print(titles, "\n")

pd_ds = pd.DataFrame(ds, columns=titles)
print(len(pd_ds))
display(pd_ds)

for k in ds:
    #display(k)
    print("\n")
    df = pd.DataFrame(k)
    print(df)

['Дата публикации', 'ФЗ', 'Цена, ₽', 'Услуги', 'Покупатель', 'Подробнее'] 

17


,Дата публикации,ФЗ,"Цена, ₽",Услуги,Покупатель,Подробнее
0,2021-03-05 14:38:14.999000+08:00,44,999733,"услуги по освещению деятельности Думы города Иркутска на сайте информационного агентства,услуги по освещению деятельности Думы города Иркутска на сайте информационного агентства,услуги по освещению деятельности Думы города Иркутска на сайте информационного агентства,услуги по освещению деятельности Думы города Иркутска на сайте информационного агентства,услуги по освещению деятельности Думы города Иркутска на сайте информационного агентства,услуги по освещению деятельности Думы города Иркутска на сайте информационного агентства,услуги по освещению деятельности Думы города Иркутска на сайте информационного агентства",ДУМА ГОРОДА ИРКУТСКА,https://zakupki.gov.ru/epz/contract/printForm/view.html?contractInfoId=64264400
1,2021-03-03 18:09:47.245000+08:00,44,200000,"Услуги по мытью потолков,Услуги по химической чистке жалюзи,Услуги по химической чистке дивана,Услуги по химической чистке кресел,Услуги по химической чистке ковров и ковровых изделий,Услуги по химической чистке тюлевых изделий,Услуги по химической чистке портьер",ЗАКОНОДАТЕЛЬНОЕ СОБРАНИЕ ИРКУТСКОЙ ОБЛАСТИ,https://zakupki.gov.ru/epz/contract/printForm/view.html?contractInfoId=64144880
2,2021-03-03 15:39:02.500000+08:00,44,216000,"Услуги местной, внутризоновой телефонной связи",АДМИНИСТРАЦИЯ ГОРОДА ИРКУТСКА,https://zakupki.gov.ru/epz/contract/printForm/view.html?contractInfoId=64208931
3,2021-03-03 14:35:21.351000+08:00,44,100000,"на оказание услуг по обеспечению проезда к месту служебной командировки и обратно, транспортному обслуживанию (бронирование, оформление и предоставление авиа билетов, железнодорожных билетов) сотрудников министерства по регулированию контрактной системы в сфере закупок Иркутской области",МИНИСТЕРСТВО ПО РЕГУЛИРОВАНИЮ КОНТРАКТНОЙ СИСТЕМЫ В СФЕРЕ ЗАКУПОК ИРКУТСКОЙ ОБЛАСТИ,https://zakupki.gov.ru/epz/contract/printForm/view.html?contractInfoId=64176772
4,2021-03-03 11:51:32.248000+08:00,44,999999,Оказание услуг по подготовке и организации размещения информационных материалов о деятельности Думы города Иркутска в федеральном общественно-политическом печатном издании,ДУМА ГОРОДА ИРКУТСКА,https://zakupki.gov.ru/epz/contract/printForm/view.html?contractInfoId=64180667
5,2021-02-26 12:30:33.934000+08:00,44,2649665,Услуги почтовой связи,АДМИНИСТРАЦИЯ ГОРОДА ИРКУТСКА,https://zakupki.gov.ru/epz/contract/printForm/view.html?contractInfoId=64091617
6,2021-02-19 12:38:30.532000+08:00,44,37200,"Средство дезинфицирующее (кожный антисептик) «Анавидин-Экспроф» 1 л,Средство дезинфицирующее (кожный антисептик) «Анавидин-Экспроф» 0,2 л,Средство дезинфицирующее (кожный антисептик) «Анавидин-Экспроф» 0,05 л",ЗАКОНОДАТЕЛЬНОЕ СОБРАНИЕ ИРКУТСКОЙ ОБЛАСТИ,https://zakupki.gov.ru/epz/contract/printForm/view.html?contractInfoId=63457860
7,2021-02-18 17:15:36.774000+08:00,44,650000,"Автобус (2),Автобус (3),Автобус (3),Автобус (1),Автобус (1),Минивэн,Минивэн,Микроавтобус,Микроавтобус,Автобус (2)",ЗАКОНОДАТЕЛЬНОЕ СОБРАНИЕ ИРКУТСКОЙ ОБЛАСТИ,https://zakupki.gov.ru/epz/contract/printForm/view.html?contractInfoId=63865584
8,2021-02-15 12:09:59.193000+08:00,44,348412,"Компьютер персональный настольный (моноблок),Ноутбук",ЗАКОНОДАТЕЛЬНОЕ СОБРАНИЕ ИРКУТСКОЙ ОБЛАСТИ,https://zakupki.gov.ru/epz/contract/printForm/view.html?contractInfoId=63738197
9,2021-02-15 12:09:58.875000+08:00,44,658356,"Сканер,Многофункциональное устройство (МФУ)",ЗАКОНОДАТЕЛЬНОЕ СОБРАНИЕ ИРКУТСКОЙ ОБЛАСТИ,https://zakupki.gov.ru/epz/contract/printForm/view.html?contractInfoId=63738091




                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                0
0                                                                                                                                                                                                                                                                                                                                                                                   